## 1. Import

In [2]:
import pandas as pd
import numpy as np

import folium
from folium import plugins

In [6]:
inputdir="C:\\zsofia\\Amsterdam\\GitHub\\komazsofi\\ecolidar_WetLand\\Data\\NoordHolland_Ex2\\"
filename="occurrence"

in_vegdb=pd.read_csv(inputdir+filename+".txt",sep='\t')

list(in_vegdb)
#in_vegdb.shape

['gbifID',
 'abstract',
 'accessRights',
 'accrualMethod',
 'accrualPeriodicity',
 'accrualPolicy',
 'alternative',
 'audience',
 'available',
 'bibliographicCitation',
 'conformsTo',
 'contributor',
 'coverage',
 'created',
 'creator',
 'date',
 'dateAccepted',
 'dateCopyrighted',
 'dateSubmitted',
 'description',
 'educationLevel',
 'extent',
 'format',
 'hasFormat',
 'hasPart',
 'hasVersion',
 'identifier',
 'instructionalMethod',
 'isFormatOf',
 'isPartOf',
 'isReferencedBy',
 'isReplacedBy',
 'isRequiredBy',
 'isVersionOf',
 'issued',
 'language',
 'license',
 'mediator',
 'medium',
 'modified',
 'provenance',
 'publisher',
 'references',
 'relation',
 'replaces',
 'requires',
 'rights',
 'rightsHolder',
 'source',
 'spatial',
 'subject',
 'tableOfContents',
 'temporal',
 'title',
 'type',
 'valid',
 'institutionID',
 'collectionID',
 'datasetID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'ownerInstitutionCode',
 'basisOfRecord',
 'informationWithheld',
 'dataGeneral

## 2. Explore the content of DwC

In [7]:
print(in_vegdb.habitat.unique())
print(in_vegdb.year.unique().shape)
print(in_vegdb.collectionID.unique().shape)

print(in_vegdb.organismQuantityType.unique())

np.set_printoptions(suppress=True)
print(in_vegdb.sampleSizeValue.unique())

# reduce for wetland areas
in_vegdb_wtl=in_vegdb[in_vegdb['habitat']=='Saltmarch'] 

in_vegdb_wtl.shape

#in_vegdb.groupby(['footprintWKT']).count().head()

['Wet pionier environment' nan 'Saltmarch' 'Grassland (not salty)'
 'Brushwood' 'Marsh' 'Heathland' 'Open water' 'Forest'
 'Fringes & tall forb vegetation' 'Dry disturbed environment']
(55,)
(1,)
['percentageCoverage']
[   25.     100.       4.       9.        nan     3.      16.      12.
     0.32     0.25     6.      10.       2.       0.5      1.53    50.
    30.     200.       0.24     1.     225.       0.45    40.       8.
  1000.     131.       2.25     2.8      0.2      1.5      5.       2.5
     1.08    18.      20.      36.      15.       4.5      0.7     24.
     0.75   176.     120.      32.       0.28    70.       2.1      0.15
     7.5     12.5      0.8    300.      17.5      6.25     1.13     1.2
    60.     400.     600.       0.81    75.      80.       1.95     1.04
    33.       1.12     7.       0.3     91.       3.2      4.05    21.
     0.6      0.49     3.5      0.4     64.       1.8      0.64     0.02
     0.16     4.8      1.75     1.4     14.      49.       3.4 

(1563, 235)

## 3. Convert into polygon based database

In [8]:
poly=in_vegdb_wtl.groupby('footprintWKT')['species'].apply(list)

groupbypoly=in_vegdb_wtl.groupby('footprintWKT')

for footprintWKT, group in groupbypoly:
    print(footprintWKT)
    print(group['species'].shape,group['habitat'].unique(),group['organismQuantityType'].unique())
    print(group[['species','organismQuantity','sampleSizeValue']])

# reduce for wetland areas
in_vegdb_PhA=in_vegdb[in_vegdb['species']=='Phragmites australis']
in_vegdb_PhA_org=in_vegdb[in_vegdb['organismQuantity']>80]

#in_vegdb_PhA_org[['footprintWKT','organismQuantity','habitat','eventID']].groupby(['footprintWKT']).head()

POLYGON((110000 555000,110000 560000,115000 560000,115000 555000,110000 555000))
(408,) ['Saltmarch'] ['percentageCoverage']
                       species  organismQuantity  sampleSizeValue
289              Festuca rubra              18.0             0.25
290         Spergularia salina               1.0             0.25
291           Elymus athericus               2.0             0.25
292            Juncus gerardii              38.0             0.25
293         Plantago coronopus               2.0             0.25
294       Agrostis stolonifera               8.0             0.25
2141             Carex extensa               3.0           100.00
2142       Oenanthe lachenalii               2.0           100.00
2143   Bolboschoenus maritimus               2.0           100.00
2144      Agrostis stolonifera               3.0           100.00
2145       Triglochin maritima               2.0           100.00
2146            Glaux maritima               2.0           100.00
2147           Ju

POLYGON((111882 558379,111882 558381,111884 558381,111884 558379,111882 558379))
(10,) ['Saltmarch'] ['percentageCoverage']
                    species  organismQuantity  sampleSizeValue
21235      Juncus maritimus              68.0              9.0
21236     Cochlearia danica               2.0              9.0
21237  Phragmites australis               2.0              9.0
21238       Cirsium vulgare               8.0              9.0
21239   Oenanthe lachenalii               2.0              9.0
21240      Senecio vulgaris               2.0              9.0
21241       Filago arvensis              38.0              9.0
21242         Carex distans               2.0              9.0
21243      Sonchus arvensis               2.0              9.0
21244         Festuca rubra               3.0              9.0
POLYGON((111889 558356,111889 558358,111891 558358,111891 558356,111889 558356))
(12,) ['Saltmarch'] ['percentageCoverage']
                    species  organismQuantity  sampleSizeVa

## 4.1. Geospatial visualization (points)

In [9]:
in_vegdb_wtl.drop(in_vegdb_wtl[in_vegdb_wtl.decimalLatitude.isnull()].index, inplace=True)
in_vegdb_wtl.drop(in_vegdb_wtl[in_vegdb_wtl.species.isnull()].index, inplace=True)

base_map = folium.Map(location=[52, 5], zoom_start=5)
marker = folium.plugins.MarkerCluster().add_to(base_map)

for name, row in in_vegdb_wtl.iterrows():
    folium.Marker([row["decimalLatitude"], row["decimalLongitude"]], popup=row["species"]).add_to(marker)

base_map

C:\Users\zsofi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\zsofi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 4.2 Geospatial visualization (polygon)